# Tiny ChatGPT
by John Robinson 02/28/2023 [Follow @johnrobinsn on Twitter](https://twitter.com/johnrobinsn)

<img src="https://www.storminthecastle.com/img/01_tiny_chatgpt_files/surfin.png"><br/>

[<img src="https://www.storminthecastle.com/img/github.svg">](https://github.com/johnrobinsn/LLMNotebooks/blob/main/nbs/01_tiny_chatgpt.ipynb) [<img src="https://www.storminthecastle.com/img/colab.svg">](https://colab.research.google.com/github/johnrobinsn/LLMNotebooks/blob/main/nbs/01_tiny_chatgpt.ipynb)

Playing around a little bit with langchain and "chain of thought reasoning" using gpt-3.  Using a prompt to guide gpt-3 to answer a fairly complex chain of related questions.  This doesn't use "ChatGPT" but rather a combination of langchain memory and the gpt-3 model from Open AI.  You can see the results below.

You can easily try this out in colab for yourselves.  You'll need a [key from OpenAI](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key).  That you can paste securely into the prompt below.  

In [ ]:
def install_dependencies():
  !pip install langchain
  !pip install openai

install_dependencies()

In [1]:
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory

As mentioned earlier, you'll need to provide an API key from OpenAI since this note book uses their gpt-3 service.

In [ ]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

In [4]:
# stuff the api key into a local environment variable so that the OpenAI module can access it
import os
os.environ['OPENAI_API_KEY']= secret

A prompt template that will feed in the history (memory) of the chat session into the LLM for it to use as context.

In [2]:
template = """
{history}
Human: {human_input}
Assistant:"""

A little class wrapper around an LLMChain.

In [5]:
class Model():
    def __init__(self):
        prompt = PromptTemplate(
            input_variables=["history", "human_input"], 
            template=template)

        self.model = LLMChain(            
            prompt=prompt,
            llm=OpenAI(temperature=0,max_tokens=2000),
            #verbose=True,
            memory = ConversationBufferMemory(),
        )

    def predict(self, human_input):
        return self.model.predict(human_input=human_input)

model = Model()

Now that the model is all setup we can try asking it a question.

In [7]:
model.predict("Who was the first man to walk on the moon?")

' Neil Armstrong was the first man to walk on the moon on July 20, 1969.'

## Chain of Thought Reasoning Example

Now we'll try a much more complex interaction consisting of a chain of questions that can refer to earlier answers.  We guide this interpretation within the prompt itself.

In [10]:
question_chain = '''
Answer the following questions, which refer to the answers of previous questions or to things you already know or understand.

Q1.  Who was the first U.S. President?
Q2. How many biological children did A1 have?
Q3. What is A2 + 5?
Q4. Who was the A3-th U.S. President?
Q5. What is the first letter of A4's surname?
Q6. Which of "Mario" and "Luigi" start with A5?
Q7. Give an array of the first A3 positive integers.
Q8. What is the second-to-last element of A7?
Q9. Remove the letter "u" from A8 spelled as a word.
Q10. Interpolate: "One {A9} all, all {A9} one."
Q11. Who wrote Don Quixote?
Q12. What is A11's first name?
Q13. Are A6 and A12 the same string? Yes/No.
Q14. Repeat A13, A3 times in a row.
Q15. Concatenate the strings A14 and A6.

Use this format:

Question ${number}: ${Question text}
Rationale: Let's think step by step. First, ${write step-by-step deductions.}
Answer: A{number} ${Answer}

Repeat this format for each question.
'''

In [11]:
print(model.predict(question_chain))


Q1. Who was the first U.S. President?
Rationale: Let's think step by step. First, we know that the first man to walk on the moon was Neil Armstrong, so this question does not refer to the answer of the previous question.
Rationale: We also know that the first U.S. President was George Washington.
Answer: A1 George Washington.

Q2. How many biological children did A1 have?
Rationale: Let's think step by step. First, we know that A1 refers to George Washington, so this question refers to the answer of the previous question.
Rationale: We also know that George Washington had no biological children.
Answer: A2 0.

Q3. What is A2 + 5?
Rationale: Let's think step by step. First, we know that A2 refers to the answer of the previous question, which was 0.
Rationale: We also know that 0 + 5 is 5.
Answer: A3 5.

Q4. Who was the A3-th U.S. President?
Rationale: Let's think step by step. First, we know that A3 refers to the answer of the previous question, which was 5.
Rationale: We also know tha

## Conclusion

Pretty exciting stuff! [Langchain](https://github.com/hwchase17/langchain) makes it pretty easy to experiment with a variety of LLMs and interaction patterns.  You should check it out.

